In [1]:
import generate
import numpy as np
import plotly.graph_objects as go

In [2]:
m1, m2 = 3.0, 1.0
L1, L2 = 1.0, 1.0
g = 9.81
X0 = np.array([np.pi / 2, np.pi / 2, 0, 0])
dt = 0.01
t_max = 20

t, X = generate.double_pendulum(m1, m2, L1, L2, g, X0, dt, t_max)
x1, y1, x2, y2 = generate.to_xy(L1, L2, X[:, 0], X[:, 1])

In [ ]:
frames = []
for i in range(1, len(t)):
    frames.append(
        go.Frame(
            data=[
                go.Scatter(
                    x=[0, x1[i], x2[i]],
                    y=[0, y1[i], y2[i]],
                    mode="lines+markers",
                    marker=dict(size=[8, 8, 8]),
                    line=dict(color="black"),
                    showlegend=False,
                ),
                go.Scatter(
                    x=x1[: i + 1], y=y1[: i + 1], mode="lines", line=dict(color="blue", width=2), name="Pendulum 1 Path"
                ),
                go.Scatter(
                    x=x2[: i + 1], y=y2[: i + 1], mode="lines", line=dict(color="red", width=2), name="Pendulum 2 Path"
                ),
            ]
        )
    )

layout = go.Layout(
    title="Double Pendulum Animation with Traces",
    xaxis=dict(range=[-2, 2], autorange=False, zeroline=False, scaleanchor="y"),
    yaxis=dict(range=[-2, 2], autorange=False, zeroline=False),
    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(
                    label="Play (Slow)",
                    method="animate",
                    args=[None, dict(frame=dict(duration=60, redraw=True), fromcurrent=True, mode="immediate")],
                ),
                dict(
                    label="Play (Normal)",
                    method="animate",
                    args=[None, dict(frame=dict(duration=10, redraw=True), fromcurrent=True, mode="immediate")],
                ),
                dict(
                    label="Play (Fast)",
                    method="animate",
                    args=[None, dict(frame=dict(duration=1, redraw=True), fromcurrent=True, mode="immediate")],
                ),
                dict(
                    label="Stop",
                    method="animate",
                    args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate")],
                ),
            ],
        )
    ],
    width=800,
    height=800,
)


initial_data = [
    go.Scatter(
        x=[0, x1[0], x2[0]],
        y=[0, y1[0], y2[0]],
        mode="lines+markers",
        marker=dict(size=[8, 8, 8]),
        line=dict(color="black"),
    ),
    go.Scatter(x=[x1[0]], y=[y1[0]], mode="lines", line=dict(color="blue", width=2), name="Pendulum 1 Path"),
    go.Scatter(x=[x2[0]], y=[y2[0]], mode="lines", line=dict(color="red", width=2), name="Pendulum 2 Path"),
]


fig = go.Figure(data=initial_data, layout=layout, frames=frames)


fig.show()

In [42]:
layout = go.Layout(title="Double Pendulum", xaxis=dict(title="x"), yaxis=dict(title="y"))

data = [
    go.Scatter(x=t, y=x1, mode="lines", name="x1"),
    go.Scatter(x=t, y=y1, mode="lines", name="y1"),
    go.Scatter(x=t, y=x2, mode="lines", name="x2"),
    go.Scatter(x=t, y=y2, mode="lines", name="y2"),
]

fig = go.Figure(data=data, layout=layout)
fig.show()

In [43]:
layout = go.Layout(
    scene=dict(xaxis_title="x1", yaxis_title="y1", zaxis_title="x2"),
    width=800,
    height=800,
)

data = [
    go.Scatter3d(
        x=x1,
        y=y1,
        z=x2,
        mode="lines",
        line=dict(color=y2, colorscale="Viridis"),
        hovertemplate="x1: %{x}<br>y1: %{y}<br>x2: %{z}<br>y2: %{line.color}<extra></extra>",
    )
]

fig = go.Figure(data=data, layout=layout)
fig.show()

In [31]:
from edmkit.util import autocorrelation

autocorr = autocorrelation(x2, 100)

layout = go.Layout(title="Autocorrelation", xaxis=dict(title="lag"), yaxis=dict(title="autocorrelation"))

data = [go.Scatter(x=np.arange(len(autocorr)), y=autocorr, mode="lines")]

fig = go.Figure(data=data, layout=layout)
fig.show()

In [44]:
from edmkit.ccm import search_best_embedding

tau_list = list(range(1, 10))
e_list = list(range(1, 10))

rho = search_best_embedding(x2[:2000], tau_list, e_list, 27)

min_L=1928


tau,e,rho
i64,i64,f64
1,4,0.999014
1,3,0.999014
1,7,0.999014
1,8,0.999014
1,6,0.999014
…,…,…
5,1,0.74205
6,1,0.74205
7,1,0.74205


In [45]:
from edmkit.embedding import lagged_embed

embedding = lagged_embed(x2, 1, 3)

layout = go.Layout(
    title="Lagged Phase Plot",
    scene=dict(xaxis_title="x(t)", yaxis_title="x(t-1)", zaxis_title="x(t-2)"),
    width=800,
    height=800,
)

data = [
    go.Scatter3d(
        x=embedding[:, 0],
        y=embedding[:, 1],
        z=embedding[:, 2],
        mode="lines",
        hovertemplate="x(t): %{x}<br>x(t-1): %{y}<br>x(t-2): %{z}<extra></extra>",
    )
]

fig = go.Figure(data=data, layout=layout)
fig.show()

In [46]:
from edmkit.simplex_projection import simplex_projection

embedding = lagged_embed(x2, 1, 5)

X = embedding[:-27]
Y = embedding[27:]

partition = 1000
t = np.arange(700) + partition

obserbations, predictions = simplex_projection(X, Y, t, partition)

# plot single dimension

layout = go.Layout(title="Simplex Projection", xaxis=dict(title="t"), yaxis=dict(title="x"))

data = [
    go.Scatter(x=t, y=obserbations[:, 0], mode="lines", name="observations"),
    go.Scatter(x=t, y=predictions[:, 0], mode="lines", name="predictions"),
]

fig = go.Figure(data=data, layout=layout)
fig.show()


# 3D plot

layout = go.Layout(
    title="Simplex Projection",
    scene=dict(xaxis_title="x(t)", yaxis_title="x(t-1)", zaxis_title="x(t-2)"),
    width=800,
    height=800,
)

data = [
    go.Scatter3d(
        x=obserbations[:, 0],
        y=obserbations[:, 1],
        z=obserbations[:, 2],
        mode="lines",
        name="observations",
        hovertemplate="x(t): %{x}<br>x(t-1): %{y}<br>x(t-2): %{z}<extra></extra>",
    ),
    go.Scatter3d(
        x=predictions[:, 0],
        y=predictions[:, 1],
        z=predictions[:, 2],
        mode="lines",
        name="predictions",
        hovertemplate="x(t): %{x}<br>x(t-1): %{y}<br>x(t-2): %{z}<extra></extra>",
    ),
]

fig = go.Figure(data=data, layout=layout)
fig.show()